In [5]:
import os
import random

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10


PROJECT = 'apple_m1_pro'
GROUP = 'keras'
JOB_TYPE = 'M1Pro'

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)


IMG_SIZE = 128
BS = 64
EPOCHS = 10

In [6]:
import wandb

In [9]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Metal device set to: Apple M1 Pro


2021-11-22 09:51:58.136042: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-22 09:51:58.136414: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


True

In [10]:
tf.test.gpu_device_name()

2021-11-22 09:52:25.257768: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-22 09:52:25.257802: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


'/device:GPU:0'

In [12]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [7]:
def hardware():
  """Detect platform and accelerator"""
  if platform.system() == "Darwin":
    if platform.processor() == "arm":
      return "Apple M1"
    else:
      return "Apple Intel"
  elif len(tf.config.list_physical_devices('GPU')) > 0:
    return "Nvidia"
  return "Linux Intel"

In [8]:
hardware()

NameError: name 'platform' is not defined

In [15]:
run = wandb.init(project=PROJECT, group=GROUP, job_type=JOB_TYPE)

In [15]:
artifact = run.use_artifact('tcapelle/apple_m1_pro/PetImages:latest', type='dataset')
artifact_dir = artifact.download()

NameError: name 'run' is not defined

In [35]:
import os
from PIL import Image

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join(artifact_dir, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [13]:
def get_data(data_dir, batch_size):
    print(f'Reading from : {Path(data_dir).absolute()}')
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        Path(data_dir),
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BS,
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        Path(data_dir),
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BS,
    )
    return train_ds, val_ds

In [14]:
train_ds, val_ds = get_data(artifact_dir, 32)

NameError: name 'artifact_dir' is not defined

In [19]:
#not supported on GPU in M1
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)
data_augmentation = lambda x: x

In [20]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [21]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=(IMG_SIZE, IMG_SIZE) + (3,), num_classes=2)

In [22]:
callbacks = [
    wandb.keras.WandbCallback(save_model=False),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=EPOCHS, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/10


2021-11-19 19:21:14.311909: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-19 19:21:14.315356: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-19 19:21:14.315483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9


126/293 [===========>..................] - ETA: 1:24 - loss: 0.6724 - accuracy: 0.6336

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


141/293 [=============>................] - ETA: 1:17 - loss: 0.6654 - accuracy: 0.6395

...

194/293 [==================>...........] - ETA: 50s - loss: 0.0551 - accuracy: 0.9786

Corrupt JPEG data: 239 extraneous bytes before marker 0xd9


293/293 [==============================] - ETA: 0s - loss: 0.0542 - accuracy: 0.9793

Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


293/293 [==============================] - 159s 541ms/step - loss: 0.0542 - accuracy: 0.9793 - val_loss: 0.4073 - val_accuracy: 0.9052


In [36]:
wandb.finish()

accuracy,▁▅▆▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▃▃▂▂▁▁▁▁
val_accuracy,▁▃▇▆██▆███
val_loss,▆▆▁▃▂▂█▁▂▂
accuracy,0.97928
best_epoch,7
best_val_loss,0.28175
epoch,9
loss,0.05418
val_accuracy,0.90517


# Old code

In [23]:
# get data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train / 255., x_test / 255.


default_config = {
    "batch_size": 64, "epochs": 10, "dropout": 0.4, "base_model": BASE_MODEL, 
    "init_lr": 0.0005, "decay": 0.96, "num_classes": N_CLASSES, "hardware": hardware(), 
    "train_size": len(x_train), "test_size": len(x_test),
    "dataset": DATASET, "img_dim": IMG_DIM, "trainable": True,
}

NameError: name 'platform' is not defined

In [ ]:
def train():
  """Run transfer learning on the configured model and dataset"""
  global IMG_DIM, N_CLASSES, DS_CACHE
  with wandb.init(project="m1-benchmark", config=default_config) as run:
    # Set global defaults when running in sweep mode
    IMG_DIM = run.config.img_dim
    N_CLASSES = run.config.num_classes
    DS_CACHE = os.path.join(tempfile.mkdtemp(), str(hash(frozenset(run.config.items()))))

    # Setup base model to transfer from, optionally fine-tune
    base_model = getattr(tf.keras.applications, run.config.base_model)(
        input_shape=(run.config.img_dim, run.config.img_dim, 3),
        include_top=False, weights='imagenet')
    base_model.trainable = run.config.trainable

    # Decay learning rate
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      run.config.init_lr, decay_steps=run.config.train_size, decay_rate=run.config.decay)

    # Compile model for this dataset
    model = tf.keras.Sequential([
      base_model,
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dropout(run.config.dropout),
      tf.keras.layers.Dense(run.config.num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'top_k_categorical_accuracy'])
    
    # Update config and print summary
    run.config.update({
      "total_params": model.count_params(),
      "trainable_params": trainable_params(model),
    })
    print("Model {}:".format(run.config.base_model))
    print("  trainable parameters:", run.config.trainable_params)
    print("      total parameters:", run.config.total_params)
    print("Dataset {}:".format(run.config.dataset))
    print("  training: ", run.config.train_size)
    print("      test: ", run.config.test_size)
    print("     shape: {}\n".format((run.config.img_dim, run.config.img_dim, 3)))
    
    # Train the model
    history = model.fit(x_train, y_train,  
                        epochs=run.config.epochs, 
                        validation_data=(x_test, y_test),
                        callbacks=[wandb.keras.WandbCallback(save_model=False)])

In [ ]:
train()